In [1]:
%config IPCompleter.greedy=True
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import Image, clear_output
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import cv2
import os
import platform

In [2]:
# Create results folder one time
if not os.path.exists("results"):
    os.mkdir("results")
    
# Include the .dll file in lib folder
# Needed for creating and exporting the video using cv2
libFolder = os.path.abspath('lib/' + platform.system())
os.environ['PATH'] = libFolder + ";" + os.environ['PATH']

In [3]:
# Load COVID-19 Data to cDate dataframe
cData = pd.read_csv('COVID-19 Data/covid_daily.csv')

In [4]:
# Main GUI - Create Labels, Buttons and Menus
title_label    = widgets.Label(value='Choose Visualization Mode')
title_menu     = widgets.Label(value='Export Video')
bubble_btn     = widgets.Button(description='Bubble')
map_btn        = widgets.Button(description='Map')
bar_btn        = widgets.Button(description='Bar')
barModes_btns  = widgets.RadioButtons(options=['Confirmed', 'Deaths'], value='Confirmed', description='Bar Mode:')
export_menu    = widgets.Dropdown(options=['Graph Type', 'Bubble', 'Map', 'Bar_Confirmed', 'Bar_Deaths'], 
                                  value='Graph Type')

# Containers And Layouts for The Components
barBtn_container  = widgets.HBox([bar_btn, barModes_btns])
btns_container    = widgets.VBox([title_label, widgets.HBox([bubble_btn, map_btn, barBtn_container])])
menu_container    = widgets.VBox([title_menu, export_menu])
main_container    = widgets.VBox([btns_container, menu_container])
outputFigure      = widgets.Output()

In [5]:
def createBubbleGraph():
    """
    Creates a bubble graph representing COVID-19 data from 22-01-2020 till 13-05-2020
    """
    return px.scatter(cData, x="Deaths", y="Recovered", size="Confirmed", title='COVID-19 Bubble Graph',
                             color="Confirmed", color_continuous_scale='Viridis', text="Country",
                             animation_frame="Date", animation_group="Country", hover_name="Country",
                             log_x=True, log_y=True, size_max=200)
    
def createMapGraph():
    """
    Creates a map graph representing COVID-19 data from 22-01-2020 till 13-05-2020
    """
    return px.choropleth(cData, locations='Country', locationmode='country names', title='COVID-19 Map Graph',
                                color='Confirmed', color_continuous_scale='Viridis', 
                                labels={'Country':'Confirmed'}, animation_frame="Date", 
                                animation_group="Country", hover_name='Country')

def createBarGraph(viewMode: str):
    """
    Creates a bar graph representing COVID-19 data from 22-01-2020 till 13-05-2020
    Bar graph has 2 viewModes: 'Confirmed' and 'Deaths'. And the User can choose between them.
    params viewMode: A string to choose which mode to view
    """
    
    # Remove the countries which have 0 confirmed cases in each day
    updatedData = cData[cData[viewMode] != 0]
    barFigure = px.bar(updatedData, x="Country", y=viewMode, title='COVID-19 Bar Graph',
                                    color_continuous_scale='Viridis', animation_frame="Date",
                                    animation_group="Country")
    
    # For updating the bars to be sorted descending
    barFigure.update_layout(xaxis={'categoryorder': 'total descending'})
    return barFigure
    
def createAllGraphs():
    """
    Creates all the graphs figures (Bubble, Map and Bar).
    Each graph is created using plotly-express (A High-level API)
    """
    
    figuresGraphs = {}                  # Dictionary contains the graphs figures
    print('Creating The Graphs...')
    figuresGraphs['Bubble']        = createBubbleGraph()
    figuresGraphs['Map']           = createMapGraph()
    figuresGraphs['Bar_Confirmed'] = createBarGraph(viewMode='Confirmed')
    figuresGraphs['Bar_Deaths']    = createBarGraph(viewMode='Deaths')
    print('All Graphs Was Created Successfully!.')
    return  figuresGraphs

figures = createAllGraphs()

Creating The Graphs...
All Graphs Was Created Successfully!.


In [6]:
def ExportGraphToMP4(graphName: str, width: int, height: int, FPS: int, Seconds: int):
    """
    Exports the given graph figure to mp4 video and save it in 'results/' folder 
    """
    global image
    video_dimensions = (width, height)
    fps = FPS
    seconds = Seconds
    fourcc = cv2.VideoWriter_fourcc(*'avc1')    
    video = cv2.VideoWriter(f"results/{graphName}Graph.mp4", fourcc, fps, video_dimensions)
    figCounter = 0
    
    print(f'Exporting {graphName} Graph to mp4 video, please wait..')
    # The Main Loop Over The Video To Draw Each Frame
    for i in range(0, seconds*fps):
        if i % 15 == 0:
            if figCounter < len(figures[graphName].frames):
                fig = go.Figure(figures[graphName].frames[figCounter].data, figures[graphName].layout)
                fig["layout"].pop("updatemenus")
                fig["layout"].pop("sliders")
                fig["layout"].pop("barmode")
                fig["layout"].pop("legend")
                img_bytes = fig.to_image(format="png")
                stream = BytesIO(img_bytes)
                image = Image.open(stream).convert("RGBA")
                stream.close()
            else:
                break

        # Draw frame.
        video.write(cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR))

        # Start/End of the frame
        # In case of 60 fps -> each figure will be drawn 18 times
        if (i+1) % 15 == 0:
            figCounter += 1
        
    video.release()
    print('Exporting finished Successfully!.')
    print(f'video saved in results/{graphName}Graph.mp4')

In [7]:
def menuChanged(_):
    """
    A Handler function for export video menu to select which type to be exported
    """
    if export_menu.value != export_menu.options[0]:
        ExportGraphToMP4(graphName=export_menu.value, width=700, height=500, FPS=60, Seconds=60)
    
def displayGraph(_, mode):
    """
    This function is responsible for displaying one the figures depending on the given mode.
    params mode: A string represents the mode of the graphs.
    """
    with outputFigure:
        clear_output()
        # Check Mode Of Bar Graph
        if mode == 'Bar':
            if barModes_btns.value == 'Confirmed':
                figures['Bar_Confirmed'].show()
            elif barModes_btns.value == 'Deaths':
                figures['Bar_Deaths'].show()
        else:
            figures[mode].show()

In [8]:
# Buttons Handler
bubble_btn.on_click(lambda x: displayGraph(x, bubble_btn.description))
map_btn.on_click(lambda x: displayGraph(x, map_btn.description))
bar_btn.on_click(lambda x: displayGraph(x, bar_btn.description))
export_menu.observe(menuChanged, names='value')
widgets.VBox([main_container, outputFigure])

Exporting Bubble Graph to mp4 video, please wait..
Exporting finished Successfully!.
video saved in results/BubbleGraph.mp4
Exporting Map Graph to mp4 video, please wait..
Exporting finished Successfully!.
video saved in results/MapGraph.mp4
Exporting Bar_Confirmed Graph to mp4 video, please wait..
Exporting finished Successfully!.
video saved in results/Bar_ConfirmedGraph.mp4
Exporting Bar_Deaths Graph to mp4 video, please wait..
Exporting finished Successfully!.
video saved in results/Bar_DeathsGraph.mp4
